<a href="https://colab.research.google.com/github/ThomasDarrieumerlou/Applied_Predictive_Cours/blob/main/NLP_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing </a>

## Assignment: K Nearest Neighbors Model for the IMDB Movie Review Dataset

For the final project, build a K Nearest Neighbors model to predict the sentiment (positive or negative) of movie reviews. The dataset is originally hosted here: http://ai.stanford.edu/~amaas/data/sentiment/

Use the notebooks from the class and implement the model, train and test with the corresponding datasets.

You can follow these steps:
1. Read training-test data (Given)
2. Train a KNN classifier (Implement)
3. Make predictions on your test dataset (Implement)

__You can use the KNN Classifier from here: https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html__

## 1. Reading the dataset

We will use the __pandas__ library to read our dataset.

#### __Training data:__
Let's read our training data. Here, we have the text and label fields. Labe is 1 for positive reviews and 0 for negative reviews.

In [ ]:
import pandas as pd

train_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_train.csv', header=0)
train_df.head()

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0


#### __Test data:__

In [ ]:
import pandas as pd

test_df = pd.read_csv('https://raw.githubusercontent.com/aws-samples/aws-machine-learning-university-accelerated-nlp/master/data/final_project/imdb_test.csv', header=0)
test_df.head()

,text,label
0,What I hoped for (or even expected) was the we...,0
1,Garden State must rate amongst the most contri...,0
2,There is a lot wrong with this film. I will no...,1
3,"To qualify my use of ""realistic"" in the summar...",1
4,Dirty War is absolutely one of the best politi...,1


## 2. Train a KNN Classifier
Here, you will apply pre-processing operations we covered in the class. Then, you can split your dataset to training and validation here. For your first submission, you will use __K Nearest Neighbors Classifier__. It is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

In [ ]:
train_df["label"].value_counts()

0    12500
1    12500
Name: label, dtype: int64

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')


# Implement this
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren',
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts):
    final_text_list=[]
    for sent in texts:

        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""

        filtered_sentence=[]

        sent = sent.lower() # Lowercase
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:

        for w in word_tokenize(sent):

            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words

        final_text_list.append(final_string)

    return final_text_list

In [ ]:
print("Processing the text fields")
train_text_list = process_text(train_df["text"].tolist())
test_text_list = process_text(test_df["text"].tolist())

print(len(train_text_list))

y_train=train_df["label"].tolist()
y_test=test_df["label"].tolist()




Processing the text fields
25000


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier


import gensim
from gensim.models import Word2Vec
### PIPELINE ###
##########################
w2v = gensim.models.Word2Vec()
pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
    #( 'text_vect', TfidfVectorizer(use_idf=True,
                                  max_features=100)),
    ('knn', KNeighborsClassifier(n_neighbors=3))
                                ])


# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=100)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
X_train = train_text_list
X_val = test_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=100)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

## 3. Make predictions on your test dataset

Once we select our best performing model, we can use it to make predictions on the test dataset. You can simply use __.fit()__ function with your training data to use the best performing K value and use __.predict()__ with your test data to get your test predictions.

In [ ]:
# Implement this

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

y_val = test_df["label"]

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[7821 4679]
 [4551 7949]]
              precision    recall  f1-score   support

           0       0.63      0.63      0.63     12500
           1       0.63      0.64      0.63     12500

    accuracy                           0.63     25000
   macro avg       0.63      0.63      0.63     25000
weighted avg       0.63      0.63      0.63     25000

Accuracy (validation): 0.6308


------------------------------------------------------------------------
Testing environment :
- Use with KNN CountVectorizer set at 10 feature

Result Table

| KNN Value | Accuracy |
| --------- | -------- |
| K = 3     | 0,51768  |
| k = 5     | 0,52288  |
| k = 7     | 0,52692  |
| k = 9     | 0.52932  |
| k = 15    | 0,53188  |

------------------------------------------------------------------------
Testing environment :
- Use with KNN CountVectorizer set at 20 feature

Result Table

| KNN Value | Accuracy |
| --------- | -------- |
| K = 3     | 0.54932  |
| k = 5     | 0.55936  |
| k = 7     | 0.56404  |
| k = 9     | 0.56828  |
| k = 15    | 0.57792  |

------------------------------------------------------------------------
Testing environment :
- Use with KNN CountVectorizer set at 50 feature

Result Table

| KNN Value | Accuracy |
| --------- | -------- |
| K = 3     | 0.6148   |
| k = 5     | 0.6276   |
| k = 7     | 0.64188  |
| k = 9     | 0.65236  |
| k = 15    | 0.66048  |

------------------------------------------------------------------------
Testing environment :
- Use with KNN CountVectorizer set at 100 feature

Result Table

| KNN Value | Accuracy |
| --------- | -------- |
| K = 3     | 0.6308   |
| k = 5     | 0.63816  |
| k = 7     | 0.65392  |
| k = 9     | 0.66056  |
| k = 15    | 0.67424  |